<a href="https://colab.research.google.com/github/yuli4ka/ReversiGene/blob/main/Reversi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [472]:
import random
import numpy as np
import math

from copy import deepcopy

In [473]:
boardSize = 6
w = 1 #white player
b = -1 #black player
white = "☺"
black = "☻"
zero = "."

invalidMove = (-1, -1)
structure = [boardSize*boardSize, 2*boardSize*boardSize,2*boardSize*boardSize, boardSize*boardSize]

In [474]:
def getSymbol(turn):
  if turn == w:
    return white
  elif turn == b:
    return black
  else:
    return zero

In [475]:
def chessCount(board, color):
  return sum(board[i].count(color) for i in range(boardSize))

In [476]:
def printBoard(board):
  print()
  for i in range(boardSize):
    for j in range(boardSize):
      print(getSymbol(board[i][j]), end = " ")
    print()
  print()
  print(black + ":" + str(chessCount(board, b)))
  print(white + ":" + str(chessCount(board, w)))
  print()

In [477]:
def generateBoard():
  board = [[0 for _ in range(boardSize)] for _ in range(boardSize)]
  board[boardSize // 2 - 1][boardSize // 2 - 1] = w
  board[boardSize // 2 - 1][boardSize // 2] = b
  board[boardSize // 2][boardSize // 2 - 1] = b
  board[boardSize // 2][boardSize // 2] = w
  return board 

In [478]:
def outOfBound(i, j):
  return i < 0 or i >= boardSize or j < 0 or j >= boardSize

In [479]:
def opposite(turn):
  return w if turn == b else b

In [480]:
def neighbourIsOpposite(board, i, j, di, dj, turn):
  if (outOfBound(i + di, j + dj)):
    return False
  return board[i + di][j + dj] == opposite(turn)

In [481]:
def checkSandwich(board, i, j, di, dj, turn):
  if di == 0 and dj == 0:
    return False
  if outOfBound(i + di, j + dj):
    return False
  if board[i + di][j + dj] == turn:
    return True
  if board[i + di][j + dj] == opposite(turn):
    return checkSandwich(board, i  + di, j + dj, di, dj, turn)
  return False

In [482]:
def turnIsValid(board, i, j, turn):
  if outOfBound(i, j):
    return False
  if board[i][j] != 0:
    return False
  for di in range (-1, 2):
    for dj in range(-1, 2):
      if neighbourIsOpposite(board, i, j, di, dj, turn):
        if checkSandwich(board, i, j, di, dj, turn):
          return True
  return False

In [483]:
def replaceNeighbours(board, i, j, di, dj, turn):
  if checkSandwich(board, i, j, di, dj, turn):
    board[i + di][j + dj] = turn
    replaceNeighbours(board, i + di, j + dj, di, dj, turn)

In [484]:
def getValidMoves(board, turn):
  return[(i, j) for i in range(boardSize) 
   for j in range(boardSize) 
    if turnIsValid(board, i, j, turn)]

In [485]:
def makeMove(board, i, j, turn):
  board[i][j] = turn
  for di in range(-1, 2):
    for dj in range(-1, 2):
      if neighbourIsOpposite(board, i, j, di, dj, turn):
        replaceNeighbours(board, i, j, di, dj, turn)
  return board

In [486]:
board = getStartBoard()
printBoard(board)


. . . . . . 
. . . . . . 
. . ☺ ☻ . . 
. . ☻ ☺ . . 
. . . . . . 
. . . . . . 

☻:2
☺:2



In [487]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [488]:
def negative(x):
  return -x

In [489]:
def evaluate(board, structre, weights, turn):
  layer = np.array(sum(board, []))
  negativeForArray = np.vectorize(negative)
  if turn == w:
    layer = negativeForArray(layer)
  for i in range(1, len(structure)):
    layer = np.append(layer, 1)
    layer = np.dot(weights[i-1], layer)
    sigmoidForArray = np.vectorize(sigmoid)
    layer = sigmoidForArray(layer)
  return layer

In [490]:
def chooseMove(board, sturcture, weights, turn):
  results = evaluate(board, structure, weights, turn)
  validMoves = getValidMoves(board, turn)
  bestMove = invalidMove
  highestValue = -1
  for coordinates in validMoves:
    valueOfCoordinates = results[boardSize * coordinates[0] + coordinates[1]]
    if valueOfCoordinates > highestValue:
      highestValue = valueOfCoordinates
      bestMove = coordinates
  return bestMove

In [491]:
def generateNetwork(structure):    
  weights = []
  for i in range(len(structure) - 1):
    weights.append(np.array([[random.uniform(-1, 1) for _ in range(structure[i] + 1)] for _ in range(structure[i + 1])]))
  return weights

In [492]:
def saveNetworks(networks, sortedNetworksTuples, previousGeneration):
  file = open("gen" + str(previousGeneration), "w")
  for networkTuple in sortedNetworksTuples:
    fitness = networkTuple[0]
    file.write("Fitness:" + str(fitness) + "\n")
    networkIndex = networkTuple[1]
    for p in range(len(networks[networkIndex])):
      for q in range(len(networks[networkIndex][p])):
        file.write(str(networks[networkIndex][p][q][0]))
        for r in range(1, len(networks[networkIndex][p][q])):
          file.write("    " + str(networks[networkIndex][p][q][r]))
        file.write("\n")
  file.close()

In [493]:
def loadNetworks(structure, previousGeneration):
  with open("gen" + str(previousGeneration), "r") as file:
    lines = file.read().splitlines()
    for i in range(len(lines) - 1, -1, -1): # removing fitness information
      if lines[i][0] == "F":
        lines.pop(i)
           
    weightsList = sum([line.split() for line in lines], [])
    # weightsList = sum([line.split("\t") for line in lines], [])
    
    weightsListIndex = 0
    networks = []

    while len(networks) < bestNetworksPopulation:
      weights = []
      for i in range(len(structure) - 1):
        layer = [[0 for _ in range(structure[i] + 1)] for _ in range(structure[i + 1])]
        for j in range(structure[i + 1]):
          for k in range(structure[i] + 1):
            layer[j][k] = float(weightsList[weightsListIndex])
            weightsListIndex += 1
          weights.append(np.array(layer))
        networks.append(weights)

  return networks

In [494]:
previousGeneration = -1
population = 30
bestNetworksPopulation = 4
mutationAmount = 0.05
generationNumber = 3
whiteCanPlay = True
blackCanPlay = True

In [495]:
if previousGeneration < 0:
    networks = [generateNetwork(structure) for _ in range(population)]
        
while previousGeneration < generationNumber:
  # repopulate
  if previousGeneration >= 0:
    print("in generation " + str(previousGeneration))
    # read networks from file if there is some previous generations
    networks = loadNetworks(structure, previousGeneration)
    bestNetworkPoplation = len(networks)

    # mutation
    for i in range(bestNetworkPoplation):
      for _ in range(5): # do 5 mutations for each neural network
        newNetwork = deepcopy(networks[i])
        for p in range(len(newNetwork)):
          for q in range(len(newNetwork[p])):
            for r in range(len(newNetwork[p][q])):
              newNetwork[p][q][r] += random.uniform(-mutationAmount, mutationAmount)
        networks.append(newNetwork)
    print("mutation")
    # crossover
    for i in range(bestNetworkPoplation):
      for j in range(bestNetworkPoplation):
        if i != j:
          for _ in range(2): # do 2 crossovers for each neural network
            newNetwork = deepcopy(networks[i])
            useNetworkJWeight = True if random.randrange(0, 1) < 0.5 else False
            for p in range(len(newNetwork)):
              for q in range(len(newNetwork[p])):
                for r in range(len(newNetwork[p][q])):
                  useNetworkJWeight = not useNetworkJWeight \
                  if random.randrange(0, 1) < 30/(8*boardSize*boardSize*boardSize*boardSize) else useNetworkJWeight
                  if useNetworkJWeight:
                    newNetwork[p][q][r] = networks[j][p][q][r]
            networks.append(newNetwork)
    print("crossover")  
  # play against each other
  print("start to play")
  previousGeneration += 1
  fitnesses = [0 for _ in range(population)]
  for i in range(population): # black
    for j in range(population): # white
      if i != j:
        turn = b
        board = generateBoard()
        whiteCanPlay = True # if white is not stuck
        blackCanPlay = True # if black is not stuck
        while whiteCanPlay or blackCanPlay:
          move = chooseMove(board, structure, networks[i if turn == b else j], turn)
          if move == invalidMove:
            if turn == w:
              whiteCanPlay = False
            if turn == b:
              blackCanPlay = False
          else :
            if turn == w:
              whiteCanPlay = True
            if turn == b:
              blackCanPlay = True
            board = makeMove(board, move[0], move[1], turn)
          # switching players
          turn = opposite(turn)
        # add to fitnesses
        # print("Generation " + str(previousGeneration) + ": network " + str(i), black, str(chessCount(board, b)),"vs.", \
        #   str(chessCount(board, w)), white + " network " + str(j))
        fitnesses[i] += chessCount(board, b)
        fitnesses[j] += chessCount(board, w)

start to play
Generation 0 Fitness:  [20.06896551724138, 17.862068965517242, 15.206896551724139, 19.655172413793103, 16.620689655172413, 18.413793103448278, 20.810344827586206, 16.086206896551722, 17.155172413793103, 19.017241379310345, 21.689655172413794, 19.655172413793103, 21.810344827586206, 20.46551724137931, 17.620689655172413, 17.051724137931036, 16.862068965517242, 19.74137931034483, 14.517241379310345, 16.93103448275862, 12.46551724137931, 18.03448275862069, 14.0, 16.017241379310345, 17.482758620689655, 18.29310344827586, 21.17241379310345, 16.844827586206897, 19.689655172413794, 17.810344827586206]
in generation 0
mutation
crossover
start to play


ValueError: ignored

In [500]:
  fitnesses = [score /((population - 1) * 2) for score in fitnesses]
  print("Generation", previousGeneration, "Fitness: ", fitnesses)

  # sort networks according to fitnesses
  networksIndices = [i for i in range(len(networks))]
  sortedNetworksTuples = [(networksIndex, fitness) for networksIndex, fitness in reversed(sorted(zip(fitnesses, networksIndices)))]

  # eliminate
  sortedNetworksTuples = sortedNetworksTuples[ : bestNetworksPopulation]

  # save best networks in the population to file
  saveNetworks(networks, sortedNetworksTuples, previousGeneration)

Generation 1 Fitness:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [501]:
board = generateBoard()
turn = b
whiteCanPlay = True
blackCanPlay = True

In [ ]:
bestNetwork = loadNetworks(structure, previousGeneration)[0]

In [ ]:
while whiteCanPlay or blackCanPlay:
  validMoves = getValidMoves()
  if len(validMoves == 0):
    print()
    if turn == w:
      whiteCanPlay = False
    if turn == b:
      blackCanPlay = False
  else:
    if turn == w:
      whiteCanPlay = True
    if turn == b:
      blackCanPlay = True
    
    printBoard()
    print("Valid moves: ", validMoves)

    if turn == b:
      while True:
        try:
          i, j = input(getSymbol(b) + ", your move: ").split()
          i, j = int(i), int(j)
          if turnIsValid(board, i, j, turn):
            break
          else:
            print("Invalid move, try again")
        except:
          print("Try again in format: 'i j'")
    else:
      i, j = chooseMove(board, structure, bestNetwork, turn)
      print(getSymbol(w]) + ", neural move: ", i, j)

    board[i][j] = turn
    for di in range(-1, 2):
      for dj in range(-1, 2):
        if neighbourIsOpposite(i, j, di, dj):
          replaceNeighbours(i, j, di, dj, turn)

  turn = opposite(turn)

print()
print("GME OVER!")
printBoard()
bb = chessCount(b)
ww = chessCount(w)
if bb == ww:
  print("Draw")
else:
  print((black if bb > ww else white), "wins")